In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
import tqdm
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['font.family'] = 'monospace'

In [3]:
gb = pd.read_csv('gambit.txt', delimiter='|', 
                  names=['slot','received_at','eligible_at','optimistic_submission','decode_duration',
                         'prechecks_duration','simulation_duration','redis_update_duration',
                         'total_duration','sim_success'])
gb.head()

,slot,received_at,eligible_at,optimistic_submission,decode_duration,prechecks_duration,simulation_duration,redis_update_duration,total_duration,sim_success
0,6280474,2023-04-2218:55:04.514129,2023-04-2218:55:04.714534,f,37443,2140,145631,15192,200407,t
1,6280508,2023-04-2219:01:53.376427,2023-04-2219:01:53.558599,f,26500,2310,144325,9039,182174,t
2,6280493,2023-04-2218:58:52.144012,2023-04-2218:58:52.33658,f,25788,2096,155831,8854,192570,t
3,6280502,2023-04-2219:00:40.170439,2023-04-2219:00:40.327336,f,9828,2202,140775,4094,156900,t
4,6280480,2023-04-2218:56:21.280912,2023-04-2218:56:21.417584,f,16412,2207,97175,20879,136674,t


In [5]:
diffs = (gb['decode_duration'] + gb['prechecks_duration'] + gb['simulation_duration'] + \
 gb['redis_update_duration'] - gb['total_duration'])
diffs.mean(), diffs.min(), diffs.max()

(-1.494059405940594, -3, 0)

In [6]:
printdf(gb)

decode\_duration & 19783 & 58904 & 280625 & 1414331 \\
prechecks\_duration & 2474 & 7166 & 202985 & 1189157 \\
simulation\_duration & 139121 & 201392 & 575156 & 2064596 \\
redis\_update\_duration & 9099 & 23170 & 598570 & 2458301 \\
\hline
\textbf{total\_duration} & \textbf{177137} & \textbf{354040} & \textbf{2049770} & \textbf{4792342} \\


In [4]:
def printdf(df):
    print('decode\_duration & ', end='')
    st = ''
    for i in [50, 75, 90, 99]:
        st += str(int(np.percentile(df['decode_duration'], i)))
        st += " & "
    print(st[:-3] + ' \\\\')
    st = ''
    print('prechecks\_duration & ', end='')
    for i in [50, 75, 90, 99]:
        st += str(int(np.percentile(df['prechecks_duration'], i)))
        st += " & "
    print(st[:-3] + ' \\\\')
    st = ''
    print('simulation\_duration & ', end='')
    for i in [50, 75, 90, 99]:
        st += str(int(np.percentile(df['simulation_duration'], i)))
        st += " & "
    print(st[:-3] + ' \\\\')
    st = ''
    print('redis\_update\_duration & ', end='')
    for i in [50, 75, 90, 99]:
        st += str(int(np.percentile(df['redis_update_duration'], i)))
        st += " & "
    print(st[:-3] + ' \\\\')
    st = ''
    print('\\hline')
    print('\\textbf{total\_duration} & ', end='')
    for i in [50, 75, 90, 99]:
        st += '\\textbf{{{}}}'.format(str(int(np.percentile(df['total_duration'], i))))
        st += " & "
    print(st[:-3] + ' \\\\')